# SCRAPING DEI PERCORSI FERROVIARI DA KAYAK

In [ ]:
#install packages for web scraping:Selenium (emulate a user on a web site) + chromium
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys #to setup the path of chromedriver
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json #to work with json file
import pprint

In [ ]:
#other stuff to work in a browser
chrome_options = webdriver.ChromeOptions() #With webdriver we can start the phantom browser, emulate the user navigation and scrape our data.
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# START SCRAPING

In [ ]:
wd = webdriver.Chrome('chromedriver',options=chrome_options) #è un oggetto che mi permette di interfacciarmi con web tramite richieste

In [ ]:
def parse_train(train,railway):  
    departure_station = ""
    arrival_station = ""
    date = ""
    departure_hour =""
    return_hour = ""
    change = ""
    price = ""
    type_of_train = ""

    try:
      if (len(train.find_elements_by_css_selector("span.top-row-station.origin")) > 0):
          departure_station = train.find_elements_by_css_selector("span.top-row-station.origin")[0].text
      if (len(train.find_elements_by_css_selector("span.top-row-station.destination")) > 0):
          arrival_station = train.find_elements_by_css_selector("span.top-row-station.destination")[0].text   
      if (len(train.find_elements_by_css_selector("span.date-text")) > 0):
          date = train.find_elements_by_css_selector("span.date-text")[0].text       
      if (len(train.find_elements_by_css_selector("div.col-field.time.depart")) > 0):
          departure_hour = train.find_elements_by_css_selector("div.col-field.time.depart")[0].text
      if (len(train.find_elements_by_css_selector("div.col-field.time.return")) > 0):
          return_hour = train.find_elements_by_css_selector("div.col-field.time.return")[0].text 
      if (len(train.find_elements_by_css_selector("div.col-field.bottom.stops-count")) > 0):
          change = train.find_elements_by_css_selector("div.col-field.bottom.stops-count")[0].text       
      if (len(train.find_elements_by_css_selector("span.price.option-text")) > 0):
          price = train.find_elements_by_css_selector("span.price.option-text")[0].text    
      if (len(train.find_elements_by_css_selector("div.G3A5-details-row")) > 0):
          type_of_train = train.find_elements_by_css_selector("div.G3A5-details-row")[0].text 
    except:
      pass
    return {'railway': railway,
            'departure station': departure_station,
            'arrival station': arrival_station, 
            'date' : date,
            'departure time': departure_hour,
            'arrival time': return_hour,
            'change': change,
            'price': price,
            'type of train': type_of_train
            }

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm

train_details1 = []
#6 ottobre 2021

#elenco delle tratte ferroviarie
list_railway = ['MIL-TRN','MIL-ROM','MIL-PAR','MIL-GVA','MIL-VIE','MIL-ZRH','MIL-PRG','MIL-NAP','MUC-STR','GVA-PAR','BRU-PAR','BRU-AMS','BRU-HAJ','AMS-PAR','BER-HAJ','BER-PAR',
'ZRH-VIE','PRG-MUC','BUD-PRG','VIE-MUC','ZAG-VIE','PAR-NCE','ROM-PAR','ROM-GVA','ROM-VIE','ROM-ZRH','ROM-MUC','ROM-TRN','FRA-PAR','FRA-AMS','FRA-MUC','FRA-PRG',
'BER-MUC','PRG-VIE','WAW-PRG','WAW-BER','AMS-HAM','HAM-BER','HAM-BRU','LON-PAR','LON-BRU','LON-AMS','MAD-VLC','PAR-LYS','TRN-LYS','MIL-LYS','BLQ-ROM','TRN-PAR',
'LYS-NCE','MIL-NCE','TRN-NCE','BCN-MAD','BCN-VLC','LON-MAN','MAD-SVQ','AMS-DUS','LON-EDI','ROM-VCE','BOL-NAP','ROM-NAP','TRN-MIL','ROM-MIL','PAR-MIL','GVA-MIL',
'VIE-MIL','ZRH-MIL','PRG-MIL','NAP-MIL','STR-MUC','PAR-GVA','PAR-BRU','AMS-BRU','HAJ-BRU','PAR-AMS','HAJ-BER','PAR-BER','VIE-ZRH','MUC-PRG','PRG-BUD','MUC-VIE',
'VIE-ZAG','NCE-PAR','PAR-ROM','GVA-ROM','VIE-ROM','ZRH-ROM','MUC-ROM','TRN-ROM','PAR-FRA','AMS-FRA','MUC-FRA','PRG-FRA','MUC-BER','VIE-PRG','PRG-WAW','BER-WAW',
'HAM-AMS','BER-HAM','BRU-HAM','PAR-LON','BRU-LON','AMS-LON','VLC-MAD','LYS-PAR','LYS-TRN','LYS-MIL','ROM-BLQ','PAR-TRN','NCE-LYS','NCE-MIL','NCE-TRN','MAD-BCN',
'VLC-BCN','MAN-LON','SVQ-MAD','DUS-AMS','EDI-LON','VCE-ROM','NAP-BOL','NAP-ROM''VIE-ZAG','VIE-STR','VIE-WAW','VIE-BEG','VIE-BUD','VIE-PRG','BRU-LIL','BRU-DUS','BRU-FRA','ZAG-MUC',
'ZAG-VCE','ZAG-BUD','BOD-LYS','BOD-NCE','BOD-TLS','BOD-GVA','LIL-PAR','LIL-AMS','LIL-DUS','LIL-LON','LYS-GVA','LYS-TLS','LYS-ZRH','NCE-GVA','TLS-BCN','HAM-DUS','HAM-HAJ',
'BER-PRG','BER-FRA','FRA-STR','FRA-ZRH','FRA-DUS','DUS-HAJ','MUC-ZRH','MUC-FRA','MUC-MIL','MUC-VCE','STR-ZRH','STR-DUS','BOL-MIL','BOL-VCE','WAW-BUD','VLC-SVQ','ZAG-VIE',
'STR-VIE','WAW-VIE','BEG-VIE','BUD-VIE','PRG-VIE','LIL-BRU','DUS-BRU','FRA-BRU','MUC-ZAG','VCE-ZAG','BUD-ZAG','LYS-BOD','NCE-BOD','TLS-BOD','GVA-BOD','PAR-LIL','AMS-LIL',
'DUS-LIL','LON-LIL','GVA-LYS','TLS-LYS','ZRH-LYS','GVA-NCE','BCN-TLS','DUS-HAM','HAJ-HAM','PRG-BER','FRA-BER','STR-FRA','ZRH-FRA','DUS-FRA','HAJ-DUS','ZRH-MUC',
'FRA-MUC','MIL-MUC','VCE-MUC','ZRH-STR','DUS-STR','MIL-BOL','VCE-BOL','BUD-WAW','SVQ-VLC']

#elenco dei viaggi diretti o con 1 cambio
for railway in list_railway:
  wd.get(f"https://www.kayak.it/trains/{railway}/2021-10-06?fs=connections=-2;transportation=~transportation_train&sort=depart_a")
  time.sleep(10) #inserisco una pausa di 5 secondi dopo essere andato sul sito, per aspettare il caricamento

  if (len(wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")) > 0):#se ci sono i cookie da accettare, accettali
    print("Click cookie law button")
    wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")[0].click()
  if (len(wd.find_elements_by_id("onetrust-accept-btn-handler")) > 0):#se ci sono i cookie da accettare, accettali
        print("Click cookie law button")
        wd.find_elements_by_id("onetrust-accept-btn-handler")[0].click()
  wd.save_screenshot("check_browser.png") #verifico che i cookies sono accettati


  for i in range(0,30):

    if (len(wd.find_elements_by_css_selector("a.page-Next.disabled")) > 0):
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      for train in tqdm(list_train):
        train_details1.append(parse_train(train,railway))
      break

    elif (len(wd.find_elements_by_css_selector("a.page-Next")) > 0):
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      for train in tqdm(list_train):
        train_details1.append(parse_train(train,railway))
      try:
        WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-Next'))).click()
      except WebDriverException:
        print("Element is not clickable")
        break
      print("Click next page")
      time.sleep(12)

    else:
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      break
print(len(train_details1))
pprint.pprint(train_details1)

100%|██████████| 15/15 [00:03<00:00,  4.21it/s]


Click next page


100%|██████████| 15/15 [00:03<00:00,  4.07it/s]


Click next page


100%|██████████| 15/15 [00:03<00:00,  3.83it/s]


...

Element is not clickable


0it [00:00, ?it/s]


Element is not clickable


100%|██████████| 2/2 [00:00<00:00,  4.10it/s]


Output streaming troncato alle ultime 5000 righe.
  'departure station': 'Torino Porta Nuova',
  'departure time': '16:03',
  'price': '30 €',
  'railway': 'TRN-MIL',
  'type of train': 'Trenitalia\n•\nHigh Speed Train'},
 {'arrival station': 'Milano Centrale',
  'arrival time': '17:45',
  'change': '',
  'date': '6/10',
  'departure station': 'Torino Porta Susa',
  'departure time': '16:05',
  'price': '31 €',
  'railway': 'TRN-MIL',
  'type of train': 'Trenitalia'},
 {'arrival station': 'Rho Rho Fiera',
  'arrival time': '17:14',
  'change': '',
  'date': '6/10',
  'departure station': 'Torino Porta Nuova',
  'departure time': '16:25',
  'price': '29 €',
  'railway': 'TRN-MIL',
  'type of train': 'Italotreno'},
 {'arrival station': 'Milano Centrale',
  'arrival time': '18:45',
  'change': '',
  'date': '6/10',
  'departure station': 'Torino Porta Nuova',
  'departure time': '16:54',
  'price': '13 €',
  'railway': 'TRN-MIL',
  'type of train': 'Trenitalia\n•\nOperato da Regionale'},


In [ ]:
import pandas as pd
ds_train_details1= pd.DataFrame(train_details1)
ds_train_details1.head(50)

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
0,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,6:40,8:35,,13 €,Trenitalia\n•\nOperato da Regionale
1,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,9:58,10:58,,31 €,Trenitalia\n•\nOperato da Frecciarossa
2,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,10:07,12:06,,13 €,Trenitalia\n•\nOperato da Regionale
3,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,10:50,11:50,,31 €,Trenitalia\n•\nOperato da Frecciarossa
4,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,13:15,15:06,,13 €,Trenitalia\n•\nOperato da Regionale
5,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,14:10,15:10,,31 €,Trenitalia\n•\nOperato da Frecciarossa
6,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,15:02,16:00,,31 €,Trenitalia\n•\nOperato da Frecciarossa
7,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,15:15,17:06,,13 €,Trenitalia\n•\nOperato da Regionale
8,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,16:02,17:00,,31 €,Trenitalia\n•\nOperato da Frecciarossa
9,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,16:15,18:06,,13 €,Trenitalia\n•\nOperato da Regionale


In [ ]:
ds_train_details1.to_csv("ds_train_details_sc1.csv", index=False) 

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm

train_details2 = []
#6 ottobre 2021

#elenco delle tratte ferroviarie
list_railway = ['MIL-VIE','MIL-PRG','MUC-STR','BRU-HAJ','ROM-PAR','ROM-MUC','MIL-NCE','TRN-NCE','LON-MAN','VIE-MIL',
'PRG-MIL','STR-MUC','BRU-HAM','NCE-TRN','VIE-STR','VIE-WAW','VIE-BEG','BRU-DUS','ZAG-VCE','ZAG-BUD','BOD-GVA','LIL-DUS','LYS-ZRH','NCE-GVA','FRA-STR','STR-ZRH','STR-DUS','WAW-BUD'
'STR-VIE','WAW-VIE','BEG-VIE','VCE-ZAG','BUD-ZAG','NCE-BOD','GVA-BOD','AMS-LIL','DUS-LIL','GVA-NCE','STR-FRA','MIL-MUC',
'ZRH-STR','DUS-STR','BUD-WAW']

#elenco dei viaggi diretti o con 1 cambio
for railway in list_railway:
  wd.get(f"https://www.kayak.it/trains/{railway}/2021-10-06?fs=connections=-2;transportation=~transportation_train&sort=depart_a")
  time.sleep(7) #inserisco una pausa di 5 secondi dopo essere andato sul sito, per aspettare il caricamento

  if (len(wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")) > 0):#se ci sono i cookie da accettare, accettali
    print("Click cookie law button")
    wd.find_elements_by_css_selector("button.Common-Widgets-Button-StyleJamButton.ui-button.size-m._id7._ihr._ihs._h-Y._im4._ii0._ihp._ihq._inw._iir._iae._iam._ifT._ifS._jV7._iiN._3-._4M._40._k-A._isy._jUK._30._43")[0].click()
  if (len(wd.find_elements_by_id("onetrust-accept-btn-handler")) > 0):#se ci sono i cookie da accettare, accettali
        print("Click cookie law button")
        wd.find_elements_by_id("onetrust-accept-btn-handler")[0].click()
  wd.save_screenshot("check_browser.png") #verifico che i cookies sono accettati


  for i in range(0,30):

    if (len(wd.find_elements_by_css_selector("a.page-Next.disabled")) > 0):
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      for train in tqdm(list_train):
        train_details2.append(parse_train(train,railway))
      break

    elif (len(wd.find_elements_by_css_selector("a.page-Next")) > 0):
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      for train in tqdm(list_train):
        train_details2.append(parse_train(train,railway))
      try:
        WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.page-Next'))).click()
      except WebDriverException:
        print("Element is not clickable")
        break
      print("Click next page")
      time.sleep(6)

    else:
      list_train = wd.find_elements_by_css_selector("div.resultInner--responsive.js-responsive-card")
      break
print(len(train_details2))
pprint.pprint(train_details2)

0it [00:00, ?it/s]


Element is not clickable


0it [00:00, ?it/s]


Element is not clickable


100%|██████████| 15/15 [00:01<00:00,  8.27it/s]


...

Click next page


100%|██████████| 15/15 [00:04<00:00,  3.30it/s]


Click next page


100%|██████████| 4/4 [00:01<00:00,  3.18it/s]
0it [00:00, ?it/s]


Element is not clickable
468
[{'arrival station': 'Stoccarda Hauptbahnhof',
  'arrival time': '2:17',
  'change': '',
  'date': '6/10',
  'departure station': 'Monaco di Baviera Hauptbahnhof',
  'departure time': '0:01',
  'price': '18 €',
  'railway': 'MUC-STR',
  'type of train': 'Deutsche Bahn'},
 {'arrival station': 'Stoccarda Hauptbahnhof',
  'arrival time': '5:35',
  'change': '',
  'date': '6/10',
  'departure station': 'Monaco di Baviera Hauptbahnhof',
  'departure time': '3:24',
  'price': '30 €',
  'railway': 'MUC-STR',
  'type of train': 'Deutsche Bahn'},
 {'arrival station': 'Stoccarda Hauptbahnhof',
  'arrival time': '7:45',
  'change': '',
  'date': '6/10',
  'departure station': 'Monaco di Baviera Hauptbahnhof',
  'departure time': '5:22',
  'price': '18 €',
  'railway': 'MUC-STR',
  'type of train': 'Deutsche Bahn'},
 {'arrival station': 'Stoccarda Hauptbahnhof',
  'arrival time': '7:45',
  'change': '',
  'date': '6/10',
  'departure station': 'Monaco di Baviera Münche

In [ ]:
ds_train_details2= pd.DataFrame(train_details2)
ds_train_details2.head()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
0,MUC-STR,Monaco di Baviera Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,0:01,2:17,,18 €,Deutsche Bahn
1,MUC-STR,Monaco di Baviera Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,3:24,5:35,,30 €,Deutsche Bahn
2,MUC-STR,Monaco di Baviera Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,5:22,7:45,,18 €,Deutsche Bahn
3,MUC-STR,Monaco di Baviera München-Pasing,Stoccarda Hauptbahnhof,6/10,5:30,7:45,,26 €,Deutsche Bahn
4,MUC-STR,Monaco di Baviera Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,5:39,8:00,,18 €,
5,MUC-STR,,,,,,,,
6,MUC-STR,,,,,,,,
7,MUC-STR,,,,,,,,
8,MUC-STR,,,,,,,,
9,MUC-STR,,,,,,,,


In [ ]:
ds_train_details2.to_csv("ds_train_details_sc2.csv", index=False)

In [ ]:
ds_train_details1 =pd.read_csv("ds_train_details_sc.csv")

In [ ]:
ds_train_details = pd.concat([ds_train_details1,ds_train_details2])

In [ ]:
ds_train_details.to_csv("ds_train_details_sc.csv", index=False) 

# ETL

In [ ]:
import pandas as pd
train_details = pd.read_csv('ds_train_details_sc.csv')
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
5891,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:18,23:08,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5892,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:48,0:50 +1,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5893,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,21:24,3:20 +1,1 cambio,18 €,Deutsche Bahn\n•\nDiversi treni\n•\nOperato da...
5894,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,21:51,3:20 +1,NaN,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5895,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,22:24,4:43 +1,3 cambi,24 €,Deutsche Bahn\n•\nDiversi treni


In [ ]:
train_details['arrival time'] = train_details['arrival time'].astype('string')
train_details['arrival time'] = train_details['arrival time'].str.rstrip(' +1').astype('string')
train_details.dtypes
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
5891,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:18,23:08,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5892,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:48,0:50,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5893,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,21:24,3:20,1 cambio,18 €,Deutsche Bahn\n•\nDiversi treni\n•\nOperato da...
5894,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,21:51,3:20,NaN,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5895,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,22:24,4:43,3 cambi,24 €,Deutsche Bahn\n•\nDiversi treni


In [ ]:
train_details['departure time'] = pd.to_datetime(train_details['departure time'], errors = 'coerce')
train_details['arrival time'] = pd.to_datetime(train_details['arrival time'].astype('string'), errors = 'coerce')
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train
5891,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,2021-08-31 20:18:00,2021-08-31 23:08:00,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5892,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,2021-08-31 20:48:00,2021-08-31 00:50:00,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5893,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,2021-08-31 21:24:00,2021-08-31 03:20:00,1 cambio,18 €,Deutsche Bahn\n•\nDiversi treni\n•\nOperato da...
5894,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,2021-08-31 21:51:00,2021-08-31 03:20:00,NaN,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...
5895,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,2021-08-31 22:24:00,2021-08-31 04:43:00,3 cambi,24 €,Deutsche Bahn\n•\nDiversi treni


In [ ]:
train_details.dtypes

railway                      object
departure station            object
arrival station              object
date                         object
departure time       datetime64[ns]
arrival time         datetime64[ns]
change                       object
price                        object
type of train                object
dtype: object

In [ ]:
train_details['duration'] = train_details['arrival time'] - train_details['departure time']

In [ ]:
train_details['departure time'] = pd.to_datetime(train_details['departure time'], errors = 'coerce').dt.time
train_details['arrival time'] = pd.to_datetime(train_details['arrival time'].astype('string'), errors = 'coerce').dt.time
train_details.head()


,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train,duration
0,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,06:40:00,08:35:00,NaN,13 €,Trenitalia\n•\nOperato da Regionale,0 days 01:55:00
1,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,09:58:00,10:58:00,NaN,31 €,Trenitalia\n•\nOperato da Frecciarossa,0 days 01:00:00
2,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,10:07:00,12:06:00,NaN,13 €,Trenitalia\n•\nOperato da Regionale,0 days 01:59:00
3,MIL-TRN,Milano Porta Garibaldi,Torino Porta Nuova,6/10,10:50:00,11:50:00,NaN,31 €,Trenitalia\n•\nOperato da Frecciarossa,0 days 01:00:00
4,MIL-TRN,Milano Centrale,Torino Porta Nuova,6/10,13:15:00,15:06:00,NaN,13 €,Trenitalia\n•\nOperato da Regionale,0 days 01:51:00


In [ ]:
train_details['duration'] = train_details['duration'].astype('string')
train_details['duration'] = train_details['duration'].str.split('s').str[-1].str.strip()
train_details['duration'] = train_details['duration'].str.split('\+').str[-1].str.strip()
train_details.tail()

,railway,departure station,arrival station,date,departure time,arrival time,change,price,type of train,duration
5891,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:18:00,23:08:00,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...,02:50:00
5892,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,20:48:00,00:50:00,1 cambio,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...,04:02:00
5893,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,21:24:00,03:20:00,1 cambio,18 €,Deutsche Bahn\n•\nDiversi treni\n•\nOperato da...,05:56:00
5894,DUS-STR,Düsseldorf Hauptbahnhof,Stoccarda Hauptbahnhof,6/10,21:51:00,03:20:00,NaN,18 €,Deutsche Bahn\n•\nInterCity Express\n•\nOperat...,05:29:00
5895,DUS-STR,Duesseldorf Intl,Stoccarda Hauptbahnhof,6/10,22:24:00,04:43:00,3 cambi,24 €,Deutsche Bahn\n•\nDiversi treni,06:19:00


In [ ]:
train_details.to_csv("ds_train_details_st.csv")

# EXTRACT OF LAT/LONG FROM CITIES

In [ ]:
import pandas as pd
ds_tratte = pd.read_csv ("tratte.csv",delimiter=";")
ds_tratte.head()

,id_tratta,partenza,arrivo,nazione partenza,nazione arrivo
0,MIL-TRN,Milano,Torino,Italia,Italia
1,MIL-ROM,Milano,Roma,Italia,Italia
2,MIL-PAR,Milano,Parigi,Italia,Francia
3,MIL-GVA,Milano,Ginevra,Italia,Svizzera
4,MIL-VIE,Milano,Vienna,Italia,Austria


In [ ]:
#identifichiamo latitudine e longitudine di tutti i progetti
import time
import requests
import json
import pprint
import pandas as pd
from tqdm import tqdm_notebook as tqdm

partenza_location = []
for partenza, project in tqdm(ds_tratte.iterrows(), total=ds_tratte.shape[0]):
  address = project['partenza']
  if(address is None):
    print(str(partenza) + "- " + project['partenza'])
  else:
    key = "insert your key"
    try:
      geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={address}"
      response = requests.get(geocode_url)
      if(response.status_code == 200):
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lng = geo['results'][0]['locations'][0]['latLng']['lng']
        partenza_location.append({
              "partenza": address,
              "lat_partenza": lat,
              "lng_partenza": lng
            })
    except Exception as e:
      print(e)
    

print((partenza_location))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/200 [00:00<?, ?it/s]

[{'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Milano', 'lat_partenza': 45.466797, 'lng_partenza': 9.190498}, {'partenza': 'Monaco', 'lat_partenza': 43.731142, 'lng_partenza': 7.419758}, {'partenza': 'Ginevra', 'lat_partenza': 46.201756, 'lng_partenza': 6.146601}, {'partenza': 'Bruxelles', 'lat_partenza': 50.838534, 'lng_partenza': 4.375434}, {'partenza': 'Bruxelles', 'lat_partenza': 50.838534, 'lng_partenza': 4.375434}, {'partenza': 'Bruxelles', 'lat_partenza': 50.838534, 'lng_partenza':

In [ ]:
ds_partenza= pd.DataFrame(partenza_location)
ds_partenza.head()

,partenza,lat_partenza,lng_partenza
0,Milano,45.466797,9.190498
1,Milano,45.466797,9.190498
2,Milano,45.466797,9.190498
3,Milano,45.466797,9.190498
4,Milano,45.466797,9.190498


In [ ]:
#identifichiamo latitudine e longitudine di tutti i progetti
import time
import requests
import json
import pprint
import pandas as pd
from tqdm import tqdm_notebook as tqdm

arrivo_location = []
for arrivo, project in tqdm(ds_tratte.iterrows(), total=ds_tratte.shape[0]):
  address = project['arrivo']
  if(address is None):
    print(str(arrivo) + "- " + project['arrivo'])
  else:
    key = "insert your key"
    try:
      geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={address}"
      response = requests.get(geocode_url)
      if(response.status_code == 200):
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lng = geo['results'][0]['locations'][0]['latLng']['lng']
        arrivo_location.append({
              "arrivo": address,
              "lat_arrivo": lat,
              "lng_arrivo": lng
            })
    except Exception as e:
      print(e)
    

print((arrivo_location))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/200 [00:00<?, ?it/s]

[{'arrivo': 'Torino', 'lat_arrivo': 45.07092, 'lng_arrivo': 7.685972}, {'arrivo': 'Roma', 'lat_arrivo': 41.893344, 'lng_arrivo': 12.483072}, {'arrivo': 'Parigi', 'lat_arrivo': 48.856506, 'lng_arrivo': 2.352133}, {'arrivo': 'Ginevra', 'lat_arrivo': 46.201756, 'lng_arrivo': 6.146601}, {'arrivo': 'Vienna', 'lat_arrivo': 48.192309, 'lng_arrivo': 16.371364}, {'arrivo': 'Zurigo', 'lat_arrivo': 47.368559, 'lng_arrivo': 8.540443}, {'arrivo': 'Praga', 'lat_arrivo': 50.08744, 'lng_arrivo': 14.421256}, {'arrivo': 'Napoli', 'lat_arrivo': 40.844116, 'lng_arrivo': 14.2423}, {'arrivo': 'Stoccarda', 'lat_arrivo': 48.778493, 'lng_arrivo': 9.180046}, {'arrivo': 'Parigi', 'lat_arrivo': 48.856506, 'lng_arrivo': 2.352133}, {'arrivo': 'Parigi', 'lat_arrivo': 48.856506, 'lng_arrivo': 2.352133}, {'arrivo': 'Amsterdam', 'lat_arrivo': 52.371009, 'lng_arrivo': 4.900112}, {'arrivo': 'Hannover', 'lat_arrivo': 52.374478, 'lng_arrivo': 9.738553}, {'arrivo': 'Parigi', 'lat_arrivo': 48.856506, 'lng_arrivo': 2.352133},

In [ ]:
ds_arrivo= pd.DataFrame(arrivo_location)
ds_arrivo.head()

,arrivo,lat_arrivo,lng_arrivo
0,Torino,45.070920,7.685972
1,Roma,41.893344,12.483072
2,Parigi,48.856506,2.352133
3,Ginevra,46.201756,6.146601
4,Vienna,48.192309,16.371364


In [ ]:
ds_partenza=ds_partenza.drop_duplicates(subset=['partenza'])
ds_arrivo=ds_arrivo.drop_duplicates(subset=['arrivo'])

In [ ]:
ds_tratte_ins= ds_tratte.merge(ds_partenza,on='partenza')
ds_tratte_fin = ds_tratte_ins.merge(ds_arrivo, on ='arrivo')

In [ ]:
len(ds_tratte_fin)

200

In [ ]:
ds_tratte_fin.to_csv('tratte.csv', index=False)